In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Generate random graphs for clusters 

In [ ]:
reps = [nx.random_regular_graph(d=2, n=10) for i in range(3)]

fig, axs = plt.subplots(1,3, figsize=(15,5))
colors = ['red', 'blue', 'green']
for i,r in enumerate(reps):
    nx.draw_circular(r, ax=axs[i], node_color=colors[i])
# plt.savefig("cluster_representative.png", transparent=True, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# generate 10 graphs per cluster that always have that patter plus some edges 

graphs = []
for r in reps:
    for i in range(10):
        A = nx.adjacency_matrix(r).todense()
        zero= np.where(A==0)
        
        ixs = np.arange(0, zero[0].size)
        np.random.shuffle(ixs)
        for _from, _to in zip(zero[0][ixs][:3], zero[1][ixs][:3]):
            A[_from, _to] = 1
            A[_to, _from] = 1
        graphs.append(A)

In [ ]:
for i in range(3):
    fig, ax = plt.subplots(2, 5, figsize=(15,10))
    for row in range(2):
        for col in range(5):
            G = nx.from_numpy_array(graphs[i*10+row*5+col])
            nx.draw_circular(G, ax=ax[row, col], node_color=colors[i])
            nx.draw_networkx_edges(reps[i], ax=ax[row, col], pos=nx.circular_layout(reps[i]), edge_color=colors[i])
    plt.savefig("members_of_cluster_"+str(i)+".png", transparent=True, dpi=200, bbox_inches='tight')
    plt.plot()
    print("----------------------")

# Code for k-means on graphs

In [ ]:
def get_representative(graphs):
    bin_graphs = []
    for g in graphs:
        g_bin = (g.copy()!=0).astype(int)
        bin_graphs.append(g_bin)
    sum_graph = np.zeros_like(g_bin)
    for g in bin_graphs:
        sum_graph += g
    return (sum_graph == len(graphs)).astype(int)

In [ ]:
def compute_distances(graphs, reps):
    distances = np.zeros((len(graphs), len(reps)))

    for i, g in enumerate(graphs):
        for j, r in enumerate(reps):
            b_g = (g!=0).astype(int)
            b_r = (r!=0).astype(int)
            diff = b_r - b_g
            how_many_plus = np.where(diff == -1)[0].size/2
            how_many_less = np.where(diff == 1)[0].size/2
            distances[i,j] = how_many_plus + 2*how_many_less
    return distances

In [ ]:
import warnings
def graph_k_means(graphs, k, max_iter=10):
    ixs = np.arange(0, len(graphs))
    np.random.shuffle(ixs)
    repres = np.array(graphs)[np.array(ixs[:k])]
    
    labels_prev = [-1]*len(graphs)
    for iter_ in range(max_iter):
        distances = compute_distances(graphs, repres)
        print(distances)
        normalized_distances = distances/np.max(distances, axis=1)[:, np.newaxis]
        similarities = 1 - normalized_distances
        print(similarities)
        kernel = similarities.dot(similarities.T)
        plt.imshow(kernel)
        plt.show()
        labels = np.argmin(distances, axis=1)
        repres = [get_representative(np.array(graphs)[np.where(labels==v)]) for v in np.unique(labels)]
        if np.all(labels == labels_prev):
            break
        print(labels)
        labels_prev = labels.copy()
    else:
        warnings.warn("The algorithm did not converge.")

In [ ]:
graph_k_means(graphs, 3)

# Inserimento in EM per inferenza grafi

In [ ]:

#un alternativa e' applicare kmeans fino a convergenza per calcolarsiil kernel
# computazionalmente e' meno efficiente e non sono sicura sia "utile", si puo' sempre provare pero'

# traccia della struttura dell'algoritmo definitivo
def clustering_inference(X, k, max_iter, add_temporal_similarity_prior=True):
    
    # fai cose preliminari 
    # il primo giro e' esterno per inizializzare la situa 
    # inferisci i grafi senza nessun prior sul kernel --> tutti si assomigliano equamente con tutti
    if add_temporal_similarity_prior:
        # il kernel iniziale e' un gaussiano con varianza abbastanza stretta

    ixs = np.arange(0, len(graphs))
    np.random.shuffle(ixs)
    repres = np.array(graphs)[np.array(ixs[:k])]
    for iter_ in range(max_iter):
        
        distances = compute_distances(graphs, repres)
        similarities = 1 - (distances/np.max(distances, axis=1)[:, np.newaxis])
        kernel = similarities.dot(similarities.T)
        if add_temporal_similarity_prior:
            # aggiungi al kernel la similarita' temporale 
        plt.imshow(kernel) # se vuoi vedere il kernel risultante 
        plt.show()
        
        # ottieni i nuovi rappresentanti
        labels = np.argmin(distances, axis=1)
        repres = [get_representative(np.array(graphs)[np.where(labels==v)]) for v in np.unique(labels)]
        
        # ottieni i nuovi grafi dato il kernel
        
        if condition: # e' stabile quando la likelihood del modello non si muove piu'
            break
    else:
        warnings.warn("The algorithm did not converge.")
